# Example
---

In [31]:
import torch
import numpy as np

from astropy        import units, constants

from pomme.plot  import plot_cube_2D
from pomme.model import TensorModel
from pomme.lines import Line
from pomme.haar  import Haar

## Download the data
---

In [5]:
# link to the model, and file name
input_link = "https://owncloud.ster.kuleuven.be/index.php/s/6mCZjZ2erTsXq5Y/download"
input_file = "model_Phantom_3D.ascii"
# link to the molecular line data, and file name
lamda_link = "https://home.strw.leidenuniv.nl/~moldata/datafiles/co.dat"
lamda_file = "co.dat"

In [6]:
# Download the model and the line data
!wget $input_link --output-document $input_file
!wget $lamda_link --output-document $lamda_file

--2023-09-27 12:41:54--  https://owncloud.ster.kuleuven.be/index.php/s/6mCZjZ2erTsXq5Y/download
Resolving owncloud.ster.kuleuven.be (owncloud.ster.kuleuven.be)... 134.58.130.75
Connecting to owncloud.ster.kuleuven.be (owncloud.ster.kuleuven.be)|134.58.130.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388714637 (371M) [application/octet-stream]
Saving to: ‘model_Phantom_3D.ascii’

model_Phantom_3D.as 100%[===================>] 370.71M  67.4MB/s    in 5.5s    

2023-09-27 12:42:00 (67.0 MB/s) - ‘model_Phantom_3D.ascii’ saved [388714637/388714637]

--2023-09-27 12:42:09--  https://home.strw.leidenuniv.nl/~moldata/datafiles/co.dat
Resolving home.strw.leidenuniv.nl (home.strw.leidenuniv.nl)... 132.229.214.179
Connecting to home.strw.leidenuniv.nl (home.strw.leidenuniv.nl)|132.229.214.179|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 444204 (434K)
Saving to: ‘co.dat’

co.dat              100%[===================>] 433.79K  1.69MB/s  

In [10]:
# Read the Phantom ascii file
(x,y,z, h, rho, vx,vy,vz, u) = np.loadtxt(input_file, skiprows=14, usecols=(0,1,2,4,5,6,7,8,9), unpack=True)

# Constants that can be read from ascii file
velocity_cte = 2.9784608e+06
density_cte  = 5.9410314e-07
energy_cte   = 8.8712277e+12

keep = np.logical_and(h>0.0, rho>0.0)

# Exclude unphysical points and points with zero abundance
x   = x  [keep]
y   = y  [keep]
z   = z  [keep]
vx  = vx [keep]
vy  = vy [keep]
vz  = vz [keep]
u   = u  [keep]
rho = rho[keep]

# Extract the number of points
npoints = len(x)

# Convert rho (total density) to abundances
nH2 = rho * density_cte * 1.0e+6 * constants.N_A.si.value / 2.02
nCO = nH2 * 1.0e-4

position = np.array((x, y, z )).T
velocity = np.array((vx,vy,vz)).T

# Convert units
position *= constants.au.si.value   # convert au to m
velocity *= velocity_cte * 1.0e-2   # convert cm/s to m/s

# Derive temperature from internal energy (assuming adiabatic heating/cooling)
gamma = 1.2
mu    = 2.381
tmp   = mu * (gamma-1.0) * u * energy_cte * 1.00784 * (units.erg/units.g * constants.u/constants.k_B).to(units.K).value

# Clamp temperatures below 2.725 K
tmp[tmp<2.725] = 2.725

# Define turbulence at 150 m/s
trb = 150.0 * np.ones(npoints)

In [34]:
# Convert point data to cartesian grid and put it ina TensorModel
haar  = Haar(position, q=8)
model = TensorModel(shape=tuple([2**(haar.q-1)] * 3), sizes=haar.xyz_L)
model['density'    ]  = haar.map_data(nCO,           interpolate=True)[-1]
model['temperature']  = haar.map_data(tmp,           interpolate=True)[-1]
model['v_turbulence'] = haar.map_data(trb,           interpolate=True)[-1]
model['velocity_los'] = haar.map_data(velocity[:,2], interpolate=True)[-1]   # velocity along z-axis


In [35]:
line = Line(
        species_name = "co",
        transition   = 0,
        datafile     = lamda_file,
        molar_mass   = 28.0
)

You have selected line:
    co(J=1-0)
Please check the properties that were inferred:
    Frequency         1.152712018e+11  Hz
    Einstein A coeff  7.203000000e-08  1/s
    Molar mass        28.0             g/mol


In [36]:
vdiff = 1500   # velocity increment size [m/s]
nfreq = 31     # number of frequencies
dd    = vdiff / constants.c.si.value * nfreq
fmin  = line.frequency - line.frequency*dd
fmax  = line.frequency + line.frequency*dd

frequencies = torch.linspace(fmin, fmax, nfreq)

In [43]:
img = line.LTE_image_along_last_axis(
    density      = model['density'     ],
    temperature  = model['temperature' ],
    v_turbulence = model['v_turbulence'],
    velocity_los = model['velocity_los'],
    frequencies  = frequencies,
    dx           = model.dx(3-1)
)

# Avoid negative values (should probably avoid these earlier...)
img = torch.abs(img)

In [46]:
plot_cube_2D(np.log10(img+1.0e-20))

interactive(children=(IntSlider(value=15, description='z', max=30), Output()), _dom_classes=('widget-interact'…

<function pomme.plot.plot_cube_2D.<locals>.plot(z)>

In [49]:
plot_spectrum(img)

interactive(children=(IntSlider(value=63, description='i', max=127), IntSlider(value=63, description='j', max=…

<function pomme.plot.plot_spectrum.<locals>.plot(i, j)>